<a href="https://colab.research.google.com/github/chen-star/llm_model_trainings/blob/main/train_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Core libs for numerical operations and plotting
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

# Libraries for text processing
import requests
import re
import string

# PyTorch specific libraries
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F

# External library for model summary and visualization
!pip install torchinfo # not installed by default in colab
from torchinfo import summary

# Matplotlib inline backend settings
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

# huggingface
from datasets import load_dataset

# [0] Download Wiki Text and Tokenization

* Download dataset from huggingface dataset `Salesforce/wikitext`

* Sample 10k wiki text and tokenize to create vocabulary lookup-table.

In [2]:
ds = load_dataset("Salesforce/wikitext", "wikitext-103-v1")


In [15]:
import re
import string

sample_size = 10000
all_text_sample = " ".join(ds['train']['text'][:sample_size])

# Split based on whitespace and punctuation
raw_words = re.split(r'[\s{}]+'.format(re.escape(string.punctuation)), all_text_sample)

# Filter out strings with length <= 1 and tokens with special characters
words = [
    word.strip().lower() for word in raw_words
    if len(word.strip()) > 1 and re.fullmatch(r'[a-zA-Z]+', word)
]

# Create vocabulary (unqiue set of words)
vocabulary = sorted(set(words))
numWords = len(words)
vocabularySize = len(vocabulary)

# Encode / Decode tables. (Token_text <-> Token_ID)
token2id = {token: idx for idx, token in enumerate(vocabulary)}
id2Token = {idx : token for idx, token in enumerate(vocabulary)}


print(f'-- This imported wiki dataset contains {numWords} words.')
print(f'-- The vocabulary contains {vocabularySize} unique words (tokens).')
print(f'-- The last 10 tokens are: \n', list(token2id.items())[:10:-1])

-- This imported wiki dataset contains 436368 words.
-- The vocabulary contains 27991 unique words (tokens).
-- The last 10 tokens are: 
 [('zygmunt', 27990), ('zweig', 27989), ('zuzanna', 27988), ('zurawik', 27987), ('zur', 27986), ('zugzwang', 27985), ('zu', 27984), ('zrinski', 27983), ('zorn', 27982), ('zoos', 27981), ('zooropa', 27980), ('zooms', 27979), ('zoomable', 27978), ('zoological', 27977), ('zookeeper', 27976), ('zooey', 27975), ('zoo', 27974), ('zoning', 27973), ('zong', 27972), ('zones', 27971), ('zone', 27970), ('zonate', 27969), ('zombies', 27968), ('zombie', 27967), ('zofia', 27966), ('zoe', 27965), ('zobrist', 27964), ('zittel', 27963), ('zirconium', 27962), ('zip', 27961), ('zionist', 27960), ('zionism', 27959), ('zinc', 27958), ('zimbabwe', 27957), ('ziltoid', 27956), ('zigzag', 27955), ('ziauddin', 27954), ('zi', 27953), ('zhu', 27952), ('zhou', 27951), ('zhongyan', 27950), ('zhishen', 27949), ('zhi', 27948), ('zhezong', 27947), ('zheng', 27946), ('zhejiang', 27945

# [1] Create pytorch Dataset and DataLoader